# RAG

In [1]:
import sys
import os
import dspy 
from common.my_settings import MySettings  
from common.utils import md
from common.llm_client_factory import LlmClientFactory
from dspy_utils.dspy_helpers import md_dspy

settings = MySettings().get()

lm = dspy.LM(
    'gpt-4.1',  # Changed from gpt-4o to gpt-3.5-turbo
    model_type='chat', 
    cache=False, 
    api_key=settings.OPENAI_API_KEY,
    temperature=0.8  
    
)

dspy.configure(lm=lm)

Getting keys from environment variables


In [2]:
# Setup

import dspy

def widget_information(expression: str):
    """Provides information about widgets like color, shape, size, cost to make, and weight."""
    return "The Widget is round, blue, and has a diameter of 5 cm and costs $100 to make. Each one weights 10kg."

def shipping_cost_calculator(weight: float, number_of_items: int) -> float:
    return weight * number_of_items * 1.1

my_tools = [widget_information]
my_tools.append(shipping_cost_calculator)

react = dspy.ReAct("question -> answer: str", tools=my_tools)

In [3]:
# Call LLM
#result = react(question="How much will it cost to create 3 widgets?")
#result = react(question="Can I please get a green widget?")
result = react(question="What is the shipping cost for 3 widgets?")

In [4]:
print(type(result))
print(result)

<class 'dspy.primitives.prediction.Prediction'>
Prediction(
    trajectory={'thought_0': "To calculate the shipping cost for 3 widgets, I need to know the weight of a single widget. I'll first retrieve information about the widget's weight.", 'tool_name_0': 'widget_information', 'tool_args_0': {'expression': 'weight of a widget'}, 'observation_0': 'The Widget is round, blue, and has a diameter of 5 cm and costs $100 to make. Each one weights 10kg.', 'thought_1': 'Now that I know each widget weighs 10kg, I can calculate the shipping cost for 3 widgets by providing the total number of items and the weight per item to the shipping cost calculator.', 'tool_name_1': 'shipping_cost_calculator', 'tool_args_1': {'weight': 10, 'number_of_items': 3}, 'observation_1': 33.0, 'thought_2': 'I have obtained the shipping cost for 3 widgets, which is 33.0. I can now finish the task and provide the final answer.', 'tool_name_2': 'finish', 'tool_args_2': {}, 'observation_2': 'Completed.'},
    reasoning=

In [5]:
md(result)

Prediction(  
    trajectory={'thought_0': "To calculate the shipping cost for 3 widgets, I need to know the weight of a single widget. I'll first retrieve information about the widget's weight.", 'tool_name_0': 'widget_information', 'tool_args_0': {'expression': 'weight of a widget'}, 'observation_0': 'The Widget is round, blue, and has a diameter of 5 cm and costs $100 to make. Each one weights 10kg.', 'thought_1': 'Now that I know each widget weighs 10kg, I can calculate the shipping cost for 3 widgets by providing the total number of items and the weight per item to the shipping cost calculator.', 'tool_name_1': 'shipping_cost_calculator', 'tool_args_1': {'weight': 10, 'number_of_items': 3}, 'observation_1': 33.0, 'thought_2': 'I have obtained the shipping cost for 3 widgets, which is 33.0. I can now finish the task and provide the final answer.', 'tool_name_2': 'finish', 'tool_args_2': {}, 'observation_2': 'Completed.'},  
    reasoning='To determine the shipping cost for 3 widgets, I first found out that each widget weighs 10kg. Using this information, I calculated the shipping cost for 3 widgets by inputting the total weight and quantity into the shipping cost calculator, which returned a total cost of 33.0.',  
    answer='The shipping cost for 3 widgets is 33.0.'  
)

In [6]:
md("**Answer** ",result.answer)
md("**Reasoning** ",result.reasoning)

md("How it got the answer...")
md(result.trajectory)
md(result.trajectory.keys())

md(len(result.trajectory)/4)
for key in result.trajectory.keys():
    md("**", key, "** ", result.trajectory[key], "** ")

**Answer** The shipping cost for 3 widgets is 33.0.

**Reasoning** To determine the shipping cost for 3 widgets, I first found out that each widget weighs 10kg. Using this information, I calculated the shipping cost for 3 widgets by inputting the total weight and quantity into the shipping cost calculator, which returned a total cost of 33.0.

How it got the answer...

{'thought_0': "To calculate the shipping cost for 3 widgets, I need to know the weight of a single widget. I'll first retrieve information about the widget's weight.", 'tool_name_0': 'widget_information', 'tool_args_0': {'expression': 'weight of a widget'}, 'observation_0': 'The Widget is round, blue, and has a diameter of 5 cm and costs $100 to make. Each one weights 10kg.', 'thought_1': 'Now that I know each widget weighs 10kg, I can calculate the shipping cost for 3 widgets by providing the total number of items and the weight per item to the shipping cost calculator.', 'tool_name_1': 'shipping_cost_calculator', 'tool_args_1': {'weight': 10, 'number_of_items': 3}, 'observation_1': 33.0, 'thought_2': 'I have obtained the shipping cost for 3 widgets, which is 33.0. I can now finish the task and provide the final answer.', 'tool_name_2': 'finish', 'tool_args_2': {}, 'observation_2': 'Completed.'}

dict_keys(['thought_0', 'tool_name_0', 'tool_args_0', 'observation_0', 'thought_1', 'tool_name_1', 'tool_args_1', 'observation_1', 'thought_2', 'tool_name_2', 'tool_args_2', 'observation_2'])

...

**observation_1** 33.0** 

**thought_2** I have obtained the shipping cost for 3 widgets, which is 33.0. I can now finish the task and provide the final answer.** 

**tool_name_2** finish** 

**tool_args_2** {}** 

**observation_2** Completed.** 

In [7]:
number_of_steps = len(result.trajectory)/4
print(int(number_of_steps)
# for i in range(number_of_steps):
#     md("### Step ", i+1)
#     md("** Action ** ", result.trajectory[f"action_{i}"])
#     md("** Action Input ** ", result.trajectory[f"action_input_{i}"])
#     md("** Observation ** ", result.trajectory[f"observation_{i}"])
#     md("** Thought ** ", result.trajectory[f"thought_{i}"])

SyntaxError: incomplete input (1494010644.py, line 8)